**Importing Libraries**

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
from time import time

torch.manual_seed(1)

**Preparing Data**

In [3]:
def prepare_sequence(seq, to_ix):
    idsx = [to_ix[w] for w in seq]
    return torch.tensor(idsx, dtype = torch.long)

In [5]:
training_data = [
    ('The dog ate the apple'.split(), ['DET', 'NN', 'V', 'DET', 'NN']),
    ('Everybody read the book'.split(), ['NN', 'V', 'DET', 'NN'])
]

word_to_ix = {}

# Making word dictionary
for sent, tag in training_data:
    for word in sent:
        if word not in word_to_ix:
            word_to_ix[word] = len(word_to_ix)
            
print('-' * 100)
print('WORD TO IX : ')
print(word_to_ix)

char_to_ix = {}

# Making char dictionary
for sent, tag in training_data:
    for word in sent:
        for char in word:
            if char not in char_to_ix:
                char_to_ix[char] = len(char_to_ix)
            
print('-' * 100)
print('CHAR TO IX : ')            
print(char_to_ix)

# Making tag dictionary
tag_to_ix = {'DET' : 0, 'NN' : 1, 'V' : 2}
            
print('-' * 100)
print('TAG TO IX : ')
print(tag_to_ix)

ix_to_tag = {0 : 'DET', 1 : 'NN', 2 : 'V'}
            
print('-' * 100)
print('IX TO TAG : ')
print(ix_to_tag)
print('-' * 100)

----------------------------------------------------------------------------------------------------
WORD TO IX : 
{'The': 0, 'dog': 1, 'ate': 2, 'the': 3, 'apple': 4, 'Everybody': 5, 'read': 6, 'book': 7}
----------------------------------------------------------------------------------------------------
CHAR TO IX : 
{'T': 0, 'h': 1, 'e': 2, 'd': 3, 'o': 4, 'g': 5, 'a': 6, 't': 7, 'p': 8, 'l': 9, 'E': 10, 'v': 11, 'r': 12, 'y': 13, 'b': 14, 'k': 15}
----------------------------------------------------------------------------------------------------
TAG TO IX : 
{'DET': 0, 'NN': 1, 'V': 2}
----------------------------------------------------------------------------------------------------
IX TO TAG : 
{0: 'DET', 1: 'NN', 2: 'V'}
----------------------------------------------------------------------------------------------------


In [6]:
def prepare_both_sequences(seq, word_to_ix, char_to_ix):
    chars = [prepare_sequence(w, char_to_ix) for w in seq]
    words = [prepare_sequence(seq, word_to_ix)]
    return chars, words

In [7]:
EMBEDDING_DIM = 6
HIDDEN_DIM = 6
CHAR_EMBEDDING = 3
CHAR_LEVEL_REPRESENTATION_DIM = 3

**Defining Model**

In [ ]:
class LSTMCharLevelAugmented(nn.Module):
    '''
    Augmented model, takes both sequence of words and char to predict tag.
    Characters are embedded and then get their own representation for each WORD.
    It is this representation that is merged with word embeddings and then fed to the sequence
    LSTM which decodes the tags.
    ARGS :
        - word_embedding_dim : embedding dimension for word
        - char_embedding_dim : embedding dimension for char
        - hidden_dim : hidden dimension for POS tag score LSTM
        - hidden_char_dim : hidden dimension for char level representation LSTM
        - vocab_size : word vocabulary size
        - charset_size : char vocabulary size
        - target_size : target set size
    '''
    def __init(self, word_embedding_dim, char_embedding_dim, hidden_dim,
              hidden_char_dim, vocab_size, charset_size, target_size):
        super(LSTMCharLevelAugmented, self).__init__()
        